In [1]:
# ========================================================================================
# THIS CODE COMPUTES THE SHUFFLE CATALOGUE OF THE G13 SAM WITH STELLAR MASS SELECTION 
# USING A NFW HALO PROFILE. ALSO IT IS COMPUTED THE CORRELATION FUNCTION FOR THE DENSITY 
# CUT 2 SAMPLE. 
# ========================================================================================

import numpy as np
import random

In [2]:
log_centralmvir = np.log10(np.load('../Data/G13_millennium/centralmvir.npy') + 1e-10) + 10
type_gal = np.load('../Data/G13_millennium/type.npy')
fofid = np.load('../Data/G13_millennium/fofid.npy')
stellarmass = np.load('../Data/G13_millennium/stellarmass.npy')
#sfr = np.load('../Data/G13_millennium/sfr.npy')
x = np.load('../Data/G13_millennium/x.npy')
y = np.load('../Data/G13_millennium/y.npy')
z = np.load('../Data/G13_millennium/z.npy')
#r_vector = np.array([x,y,z]).T
#del x,y,z

In [3]:
h = 0.73
def r200 (logM):
    t = (h*10**logM)**(1./3)
    return  (1.63e-2) * t * 1e3 # pc/h
        
def conc (log_Mh,z):
    c1 = 2e13
    t1 = (10.**(log_Mh) /c1)**(-0.13)
    return (11./(1+z)) * t1

def delta_c (c):
    return (200./3)* (c**3) /(np.log(1+c) -c/(1+c))

def g(x):
    return np.log(1+x) - x/(1+x)

tt = np.argsort(fofid)
fofid = fofid[tt]
log_centralmvir = log_centralmvir[tt]
stellarmass = stellarmass[tt]
type_gal = type_gal[tt]
x = x[tt]
y = y[tt]
z = z[tt]

idx_cen = np.where(type_gal == 0)[0]
halomass_cen = log_centralmvir[idx_cen]
fo,ngal = np.unique(fofid, return_counts=True)
del fo, log_centralmvir, type_gal

x_cen = x[idx_cen]
y_cen = y[idx_cen]
z_cen = z[idx_cen]
del x,y,z
#del idx_cen

import time
from scipy.interpolate import UnivariateSpline
start_time = time.time()

NGAL = np.sum(ngal)
stellarmass_shuffle = np.zeros(NGAL)
stellarmass_shuffle[idx_cen] = stellarmass[idx_cen] 
type_gal = np.ones(NGAL, dtype=np.int16)

i = 0
for n in ngal:
    type_gal[i] = 0
    i += n

In [ ]:
h = 0.73
def r200 (logM):
    t = (h*10**logM)**(1./3)
    return  (1.63e-2) * t * 1e3 # pc/h
        
def conc (log_Mh,z):
    c1 = 2e13
    t1 = (10.**(log_Mh) /c1)**(-0.13)
    return (11./(1+z)) * t1

def delta_c (c):
    return (200./3)* (c**3) /(np.log(1+c) -c/(1+c))

def g(x):
    return np.log(1+x) - x/(1+x)

tt = np.argsort(fofid)
fofid = fofid[tt]
log_centralmvir = log_centralmvir[tt]
stellarmass = stellarmass[tt]
type_gal = type_gal[tt]
x = x[tt]
y = y[tt]
z = z[tt]

idx_cen = np.where(type_gal == 0)[0]
halomass_cen = log_centralmvir[idx_cen]
fo,ngal = np.unique(fofid, return_counts=True)
del fo, log_centralmvir,

x_cen = x[idx_cen]
y_cen = y[idx_cen]
z_cen = z[idx_cen]
del x,y,z
#del idx_cen

import time
from scipy.interpolate import UnivariateSpline
start_time = time.time()

NGAL = np.sum(ngal)
stellarmass_shuffle = np.zeros(NGAL)
stellarmass_shuffle[idx_cen] = stellarmass[idx_cen] 

#i = 0
#for n in ngal:
#    type_gal[i] = 0
#    i += n

display_step = NGAL/1000

cat_data = np.zeros((NGAL,4), dtype=np.float32)
NHALOS = len(halomass_cen)
i,j = 0,0

# FALTA AGREGAR SI SON SATELITES O NO
while i < NHALOS:
    
    if i % display_step == 0:
        print "Step: %i Time: %3fs" % (i, (time.time() - start_time))
    
    halo_ngal = ngal[i]
    cat_data[j] = np.array([x_cen[i], y_cen[i], z_cen[i],halomass_cen[i]])
    j += 1
    
    if halo_ngal == 1:
        i += 1
        continue
    
    else:
        c = 13.981 #conc(halomass_cen[i], 0)
        x0 = np.linspace(1e-5,1,10)
        y0 = np.zeros(10)
        for k in range(len(x0)):
            r_norm0 = 1e-10
            cmf0 = g(c*r_norm0)/g(c)
            while x0[k] > cmf0:
                r_norm0 += 0.01
                cmf0 = g(c*r_norm0)/g(c) 
            y0[k] = r_norm0    

        spl = UnivariateSpline(x0, y0)
        rr = np.random.random_sample(halo_ngal-1)
        r_norm = spl(rr)

        r = r_norm * r200(halomass_cen[i])
        phi = np.random.random_sample(halo_ngal-1) * 2*np.pi
        theta = np.random.random_sample(halo_ngal-1) * np.pi

        X = r*np.sin(theta)*np.cos(phi)
        Y = r*np.sin(theta)*np.sin(phi)
        Z = r*np.cos(theta)

        x_sat = x_cen[i] + X/1e6
        y_sat = y_cen[i] + Y/1e6
        z_sat = z_cen[i] + Z/1e6
        
        halomass_array = np.ones(len(x_sat))*halomass_cen[i]

        #fn = j + halo_ngal
        cat_data[j+1:(j+1) +(halo_ngal-1),:] = np.array([x_sat, y_sat, z_sat, halomass_array]).T
        stellarmass_shuffle[j:fn] = stellarmass[j:fn]

        j += halo_ngal
        i += 1
        #if i % display_step == 0:
        #    print "Step: %i Time: %3fs" % (i, (time.time() - start_time))
        
del r_norm, r, phi,theta,X,Y,Z,x_sat,y_sat,z_sat
del x_cen,y_cen,z_cen
del halomass_cen, halomass_array, stellarmass

In [ ]:
np.save('../Data/G13_millennium/cat_data.npy',cat_data)
np.save('../Data/G13_millennium/stellarmass_shuffle_nfw2.npy', stellarmass_shuffle)


In [27]:
import numpy as np
data = np.load('../Data/G13_millennium/cat_data.npy')
#stellar = np.load('../Data/G13_millennium/stellarmass_shuffle_nfw2.npy')

logm_min = 10.
logm_max = 15.
NBIN = 60
mass_index = ((data[:,3] - logm_min)/(logm_max - logm_min) * NBIN).astype(int)
mask = (mass_index >= 0) & (mass_index < NBIN)
type_gall = type_gal[mask]

In [38]:
data2 = np.concatenate((data2,np.array([type_gall]).T),axis=1)

In [45]:
type_gall

array([0, 1, 1, ..., 0, 0, 0], dtype=int16)

In [53]:
np.where(type_gall == 0)[0][15]

44962

In [56]:
data2[44959:44972]

array([[ 266.09606934,  170.43177795,  393.21588135,   14.94154072,    1.        ],
       [ 266.83178711,  171.47796631,  392.77844238,   14.94154072,    1.        ],
       [ 266.35507202,  169.75010681,  391.68249512,   14.94154072,    1.        ],
       [ 435.36685181,   69.44320679,  468.72521973,   14.85103416,    0.        ],
       [ 435.32977295,   69.48150635,  468.92053223,   14.88376522,    1.        ],
       [ 435.49151611,   69.57315063,  469.18405151,   14.88376522,    1.        ],
       [ 435.4574585 ,   69.27435303,  468.91036987,   14.88376522,    1.        ],
       [ 435.86566162,   69.65499878,  469.25320435,   14.88376522,    1.        ],
       [ 434.70303345,   69.60211182,  467.83303833,   14.88376522,    1.        ],
       [ 435.33618164,   69.4201355 ,  468.70831299,   14.88376522,    1.        ],
       [ 435.19104004,   69.16967773,  468.99679565,   14.88376522,    1.        ],
       [ 435.43859863,   70.30209351,  468.33898926,   14.88376522,    1.   

In [34]:
a = np.array([[1,2],[3,4]])
b = np.array([[5,6]])
np.concatenate((a,b.T), axis=1)

array([[1, 2, 5],
       [3, 4, 6]])

In [ ]:
cen_pos = np.zeros(NGAL,dtype=np.int16)
i,j = 0,0
while i < NGAL:
    cen_pos[i] = 1
    i += ngal[j]
    j += 1

In [ ]:
mask = (mass_index >= 0) & (mass_index < NBIN)
mass_index = mass_index[mask]        # mass indexes in interest region
cen_pos = cen_pos[mask]              # position of centrals in interest regions
idx_cen = np.where(cen_pos != 0)[0]
mass_index_cen = mass_index[idx_cen] # mass index of center galaxies
stellarmass_shuffle = stellarmass_shuffle[mask]
np.save('../Data/G13_millennium/stellarmass_shuffle_nfw2.npy', stellarmass_shuffle)

TOT = len(mass_index) # number of galaxies
del mass_index, cen_pos, stellarmass_shuffle
fofid = fofid[mask]
cat_data = cat_data[mask]
#sfr = sfr[mask]

del mask

In [ ]:
%%time
N,j,k = 0,0,0
V = np.zeros((TOT,3))
len_data = TOT

for i in range(NBIN):
    central_positions = idx_cen[mass_index_cen == i]#np.where((type_gal == 0) & (mass_index == i))[0]
    num_halos = len(central_positions)
    shuffled_pos = np.array(random.sample(central_positions, num_halos))
    for j in range(num_halos):
        
        pos_i = shuffled_pos[j]
        init_mass = cat_data[shuffled_pos[0]]
        if j == num_halos - 1: 
            pos_f = shuffled_pos[0]
            cat_data[pos_i][3] = init_mass
        
        else: 
            pos_f = shuffled_pos[j+1]
            cat_data[pos_i][3] = cat_data[pos_f][3]
            
        v_dir = cat_data[pos_f][:3] - cat_data[pos_i][:3]     
        cond = True

        while cond:
            V[pos_i + N] = v_dir 
            N += 1  # Recorre el halo
            if pos_i + N == len_data: break
            cond = (fofid[pos_i+N] == fofid[pos_i])
            k += 1 # Recorre todas las galaxias
            
        N = 0
    

In [ ]:
%%time
def isout (r,vr):
    if r + vr < 0:
        return 1
    elif r + vr > 500: 
        return -1
    else:
        return 0

Lbox = 500.    
for i in range(len_data):
    
    x,y,z = cat_data[i,:3]
    dx,dy,dz  = V[i]

    cat_data[i][0] = x + dx + isout(x,dx)*Lbox
    cat_data[i][1] = y + dy + isout(y,dy)*Lbox
    cat_data[i][2] = z + dz + isout(z,dz)*Lbox
    
del V    

In [ ]:
np.save('../Data/G13_millennium/SAM_shuffle_nfw.npy', cat_data)

In [16]:
len(stellar),len(data),len(type_gal)

(26909106, 26909106, 27365836)

In [ ]:
import numpy as np
import corr_function
import matplotlib.pyplot as plt

model = 'G13_millennium'
path = "/home/esteban/Escritorio/Esteban HOD/Data/" + model
#r = np.load(path + '/r_nfw.npy')

x = cat_data[:,0].astype('float64')
y = cat_data[:,1].astype('float64')
z = cat_data[:,2].astype('float64')

DD, RR = corr_function.xi_ll(x,y,z,'Mstell', 2, path,Shuffle = True )

In [ ]:
Dmin = -2  
Dmax = 2. 
NBIN = 40
Bw = (Dmax - Dmin)/NBIN
bins = np.array([Dmin + Bw*i for i in range(NBIN)]) + Bw*0.5

model = 'G13_millennium'
xi_path_shuffle = "../Data/" + model + '/xi_sam' + '/Shuffle' + '/Stellar Mass'
np.savetxt(xi_path_shuffle + '/xi_shuffle_d2_nfw2.txt', np.array([bins, xi, DD, RR]).T) 

In [ ]:
xi = DD/RR - 1 
logxi = np.log10(xi)

plt.plot(bins, logxi, 'k-')
plt.show()